In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb # XGBoost implementation
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Читаем входные файлы с данными

In [2]:
data = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
target = pd.read_csv('train_target.csv', header = None)

In [4]:
features = [x for x in data.columns]
cat_features = [x for x in data.select_dtypes(include=['object']).columns]
num_features = [x for x in data.select_dtypes(exclude=['object']).columns]

In [69]:
num_features

['age',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

Для каждого клиента подсчитываем различные mcc_code

In [5]:
ntrain = data.shape[0]
ntest = target.shape[0]
train_test = pd.concat((data[features], test[features])).reset_index(drop=True)
for c in range(len(cat_features)):
    train_test[cat_features[c]] = train_test[cat_features[c]].astype('category').cat.codes

train_x = train_test.iloc[:ntrain,:]
test_x = train_test.iloc[ntrain:,:]

data

In [63]:
data = pd.get_dummies(data)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,26,901,1,999,0,1.4,94.465,-41.8,4.961,5228.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,46,208,2,999,0,1.4,93.444,-36.1,4.963,5228.1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,49,131,5,999,0,1.4,94.465,-41.8,4.864,5228.1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,31,404,1,999,0,-2.9,92.469,-33.6,1.044,5076.2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,42,85,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,56,157,5,999,0,-0.1,93.200,-42.0,4.021,5195.8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,31,202,2,999,0,1.4,94.465,-41.8,4.959,5228.1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,47,335,2,999,0,1.4,93.918,-42.7,4.962,5228.1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,27,101,1,999,1,-1.8,92.893,-46.2,1.327,5099.1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,35,91,1,999,1,-1.8,92.893,-46.2,1.327,5099.1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [6]:
target = target.reset_index(drop=True)

In [21]:
target

,0
0,1
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [96]:
test.shape

(13593, 21)

In [9]:
xgdmat = xgb.DMatrix(train_x, target) #Create our DMatrix to make XGBoost more efficient



In [17]:
num_rounds = 1000
param_test2 = {
 'max_depth':[4,5],
 'min_child_weight':[4,6],
    'gamma': [0, 0.2, 0.5, 0.9, 0.99, 1, 1.1]
}
gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_x, target)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

IndexError: too many indices for array

In [116]:
params = {
'min_child_weight': 1,
'eta': 0.05,
'colsample_bytree': 0.5,
'max_depth': 14,
#'n_estimators': 62,
'subsample': 0.8,
'alpha': 1,
'gamma': 0.95,
'silent': 1,
#'verbose_eval': True,
'eval_metric': 'auc',
'seed': 42}

In [117]:


# Grid Search CV optimized settings
num_rounds = 2000
bst = xgb.train(params, xgdmat, num_boost_round = num_rounds)

In [44]:
clf = xgb.XGBClassifier()

In [55]:
parameters = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
    }
clf = GridSearchCV(clf, params, n_jobs=1, cv=2)

In [64]:
clf.fit(data, target)

IndexError: too many indices for array

In [25]:
test_xgb = xgb.DMatrix(test_x)

In [118]:
resukt = bst.predict(test_xgb)

In [119]:
resukt
submission = pd.DataFrame()
submission['Prediction'] = resukt
submission.to_csv('resukt8')

In [76]:
submission = pd.read_csv("random.csv")

In [77]:
xx = pd.concat([test_x, a], axis=1)
b = test_x.index
b=pd.DataFrame(b)

NameError: name 'a' is not defined

In [5]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [6]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [7]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train.values[:, 0])

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=13, subsample=1.0, verbose=0,
              warm_start=False)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [10]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [11]:
result.to_csv('baseline_a.csv', index=False)

In [79]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.metrics import mean_absolute_error

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
target = pd.read_csv('train_target.csv')
test['loss'] = np.nan
train = pd.concat([train,target], axis=1)
joined = pd.concat([train, test])



In [80]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))



In [81]:
if __name__ == '__main__':
    for column in list(train.select_dtypes(include=['object']).columns):
        if train[column].nunique() != test[column].nunique():
            set_train = set(train[column].unique())
            set_test = set(test[column].unique())
            remove_train = set_train - set_test
            remove_test = set_test - set_train

            remove = remove_train.union(remove_test)
            def filter_cat(x):
                if x in remove:
                    return np.nan
                return x

            joined[column] = joined[column].apply(lambda x: filter_cat(x), 1)
            
        joined[column] = pd.factorize(joined[column].values, sort=True)[0]

    train = joined[joined['loss'].notnull()]
    test = joined[joined['loss'].isnull()]

    shift = 200
    y = np.log(train['loss'] + shift)
    ids = test['id']
    X = train.drop(['loss', 'id'], 1)
    X_test = test.drop(['loss', 'id'], 1)

KeyError: 'id'

In [78]:



if __name__ == '__main__':
    for column in list(train.select_dtypes(include=['object']).columns):
        if train[column].nunique() != test[column].nunique():
            set_train = set(train[column].unique())
            set_test = set(test[column].unique())
            remove_train = set_train - set_test
            remove_test = set_test - set_train

            remove = remove_train.union(remove_test)
            def filter_cat(x):
                if x in remove:
                    return np.nan
                return x

            joined[column] = joined[column].apply(lambda x: filter_cat(x), 1)
            
        joined[column] = pd.factorize(joined[column].values, sort=True)[0]

    train = joined[joined['loss'].notnull()]
    test = joined[joined['loss'].isnull()]

    shift = 200
    y = np.log(train['loss'] + shift)
    ids = test['id']
    X = train.drop(['loss', 'id'], 1)
    X_test = test.drop(['loss', 'id'], 1)
    
    RANDOM_STATE = 2016
    params = {
        'min_child_weight': 1,
        'eta': 0.01,
        'colsample_bytree': 0.5,
        'max_depth': 12,
        'subsample': 0.8,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'verbose_eval': True,
        'seed': 3233
    }

    xgtrain = xgb.DMatrix(X, label=y)
    xgtest = xgb.DMatrix(X_test)

    model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)

    prediction = np.exp(model.predict(xgtest)) - shift

    submission = pd.DataFrame()
    submission['loss'] = prediction
    submission['id'] = ids
    submission.to_csv('sub_v.csv', index=False)

OSError: File b'train_target' does not exist

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [90]:
a = pd.read_csv('resukt2')

In [92]:
a['Prediction'][a['Prediction']<0]=0

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [93]:
a.to_csv('resuct6', index=False)

In [87]:
a

,Id,Prediction
0,0,0.003835
1,1,0.689837
2,2,0.040119
3,3,0.001412
4,4,0.000000
5,5,0.112084
6,6,0.000000
7,7,0.000581
8,8,0.002568
9,9,0.000000


In [105]:
data = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
target = pd.read_csv('train_target.csv', header=None)

In [106]:
data = pd.get_dummies(data, )
test = pd.get_dummies(test)

In [107]:
target.shape

(27595, 1)

In [111]:
params = [{                   
        'gamma' : [0.8,0.85, 0.9, 0.92, 0.991,0.995, 0.95],'eta': [0.01,0.5,0.1], 'seed':[0], 'subsample': [0.5], 'colsample_bytree': [0.5], 
             'objective': ['reg:linear'], 'max_depth': [1,6,3,12,9], 'min_child_weight':[1,2,3], 
}]
clf = GridSearchCV(xgb.XGBClassifier(learning_rate=0.3), params) 
best = clf.fit(data, target)

IndexError: too many indices for array